# Big project

## Submission

Submit solutions to https://forms.office.com/e/WexY7YraJb.

1.   Upload code in .ipynb file
2.   Upload a csv containing three columns: 
*    "ID": the ID of the instance (1, 2, 3, ...)
*    "OBJ": the objective function value obtained
*    "TIME": the execution time in seconds.

## Evaluation

1.   Gap w.r.t. optimal solutions
2.   Runtimes. Must be under 10 minutes for every instance. Execution times will be re-examined on a random basis.

## Deadline

12/01/2022 23:59 CET

## Other

*   4 lab points just if you deliver something that works
*   10 points based on the quality of method
*   NO pre-coded libraries, 
*   NO genetic algorithms 
*   NO neural networks
*   groups of max 3 students




## Biogas plants location

An association of $n$ farmers wants to open $p$ plants to produce energy from biogas. 
Each plant will be opened at a farm of a member of the association and will be powered with corn chopping purchased from the farm itself or from other neighboring farms.

Each farm $i$ can provide at most $c_i$ tons of corn chopping, with a percentage of dry matter $a_i$. As you may know, dry matter is the key component of corn chopping used for biogas production. In order to maintain the quality of produced energy, each plant must burn a mixture of corn chopping with a percentage of dry matter between $k_{min}$ and $k_{max}$. 

At most one plant can be located in each farm, and every farm can sell its corn chopping to one and only one plant.

Each farm $i$ is located at coordinates $x_i$ and $y_i$, representing respectively its latitude and longitude, and the cost of moving corn chopping from a farm $i$ to a farm $j$ is proportional to the euclidean distance between the two farms (it does not depend on the actual quantity moved, since the trucks used for this transportations are sufficiently big). 

Under such conditions, every plant produces $Q$ kWh of energy per ton of corn chopping burned. The energy produced by each plant will be fed into the national electricity system, at a unitary price of $b$ (€/kWh). Moreover, due to state regulations, each plant must not produce more than $M$ kWh of energy.

You must locate $p$ plants among the available farms and assign the farms that will supply each plant, with the goal of maximizing the total revenues of the association.

### Sets
*   $I$ = set of farms

### Parameters
*   $n$ = number of farms   
*   $p$ = number of plants to locate
*   $b$ = revenue per unit of energy (€/kWh)
*   $M$ = max energy production (kWh)
*   $Q$ = energy produced by a ton of corn chopping (kWh/t)
*   $k_{min} (k_{max})$ = min (max) percentage of dry matter for fermentation
*   $a_i$ = percentage of dry matter in chopping from farm $i \in I$
*   $c_i$ = tons of corn chopping available for each $i \in I$ (t)
*   $x_i, y_i$ = coordinates of farm $i \in I$

In [31]:
# Clean files if present
!rm instance_1.json
!rm instance_1.txt

# Download instance_1 (both instance and solution) directly from Github
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_1.json
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_1.txt

rm: cannot remove 'instance_1.json': No such file or directory
rm: cannot remove 'instance_1.txt': No such file or directory
--2023-01-11 22:52:22--  https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 572 [text/plain]
Saving to: ‘instance_1.json’

instance_1.json     100%[===================>]     572  --.-KB/s    in 0s      

2023-01-11 22:52:23 (18.8 MB/s) - ‘instance_1.json’ saved [572/572]

--2023-01-11 22:52:23--  https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubu

In [29]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import json
import mip
import math
import numpy as np
# Reads a .json instance and returns it in a dictionary
def load_instance(filename):
  with open(filename, 'r') as f:
    data = json.load(f)
  return data

# Reads a .txt result and returns it
def load_result(filename):
  with open(filename, 'r') as f:
      result = f.read()
  return float(result)

def solve(instance):
    # Brief explanation of the proposed solution:
    # after various attempts, we ended up with a solution which tries to obtain the best result with a good time consumption,
    # without exceeding the maximum time (10 minutes) in any instance.
    # In the objective function, the distances among farms have been computed using a fast matrix multiplication algorithm.
    # Regarding the optimization of the problem, we noticed that for the instances of small sizes the problem could be solved
    # optimally and quickly with binary variables, so we decided to opt for this type of resolution if the model is sufficiently simple.   
    # As the model started to increase in the number of values, we implemented an algorithm with the following logic:
    # 1) we run a first optimization of the problem with the LP relaxation which gives us the upper bound.
    # 2) the binary vector z that indicates whether a plant is opened contains fractional values due to the relaxation,
    #    so we iteratively add a new constraint that pushes the highest fractional value to 1 (open plant) and
    #    perform a new optimization every time, in order to be as accurate as possible
    # 3) we now focus on the binary matrix w that states whether farm i sells to j, similarly to step 2) we add new
    #    constraints to push the fractional values to 1 or 0 if they are below a small epsilon
    # At this point we obtain an integer solution (+/- an epsilon since w and z are floats) that is an approximation
    # of the optimal LP solution
    # NOTE: We observed that by adding a pool of cuts (generated automatically) every time we optimize the model, it helps in
    # reaching a better integer solution, however as the model gets larger we don't observe a meaningful increase in the
    # quality of the solution despite a considerable time penalty. Therefore we decided to generate cuts only on
    # medium size models, obtaining a better solution, while on big models we prioritized execution time


    # maximum energy produced by each plant
    M = instance['M']

    # energy produced per ton of corn chopping and unitary price
    Q = instance['Q']
    b = instance['b']

    # capacity and dry matter percentage of each farm
    c = instance['c']
    a = instance['a']

    # minimum and maximum dry matter percentage for each plant
    kmax = instance['kmax']
    kmin = instance['kmin']

    # number of farms and plants
    n = instance['n']
    p = instance['p']

    N = range(n)
    P = range(p)

    # coordinates of farms
    points = instance['points']

    # create the MIP model
    m = mip.Model()

    # decision variables
    # z[i] = 1 if farm i has a plant
    z = {i: m.add_var(var_type=mip.BINARY) for i in N}

    # q[i,j] = tons of corn chopping sold by farm i to plant j
    q = {(i, j): m.add_var() for i in N for j in N}

    # w[i,j] = 1 if farm i sells to plant j
    w = {(i, j): m.add_var(var_type=mip.BINARY) for i in N for j in N}

    # constraints
    # A1 - one farm can sell its corn chopping to at most one plant
    for i in N:
        m.add_constr(mip.xsum(w[i, j] for j in N) <= 1)

    # A2 - only sell to plants
    for j in N:
        m.add_constr(mip.xsum(w[i, j] for i in N) <= n * z[j])

    # A3 - linking constraint for w[i,j] and q[i,j] and maximum farm capacity
    for i in N:
        for j in N:
            m.add_constr(q[i, j] <= w[i, j] * c[i])

    # A4 - number of plants
    m.add_constr(mip.xsum(z[i] for i in N) == p)

    # A5 - plant j must not produce more energy than allowed
    for j in N:
        m.add_constr(mip.xsum(q[i, j] for i in N) <= M / Q)

    # A6 - dry matter percentage of corn chopping burned is within limits
    for j in N:
        m.add_constr(mip.xsum(a[i] * q[i, j] for i in N) >= kmin * mip.xsum(q[i, j] for i in N))
        m.add_constr(mip.xsum(a[i] * q[i, j] for i in N) <= kmax * mip.xsum(q[i, j] for i in N))

    # objective function: maximize total revenues of the association
    # distance matrix
    matrix = np.array(points)
    xy = matrix @ matrix.T
    x2 = xy.diagonal()[:, np.newaxis]
    dis = np.abs(x2 + x2.T - 2. * xy) ** 0.5

    price_per_tons = b * Q
    m.objective = mip.maximize(mip.xsum(mip.xsum(price_per_tons * q[i, j] - w[i, j] * dis[i][j] for j in N) for i in N))

    # if the model is sufficiently simple, solve it optimally
    if(n<=12):
        m.optimize()
        return m.objective_value
    


    status = m.optimize(relax=True)
    # print(f"Upperbound: {m.objective_value}")

    # add cuts to medium sized models (see explanation above)
    max_c = 8192
    if(n<=80):
        cp = m.generate_cuts(max_cuts=max_c)
        m += cp
        m.optimize(relax = True)


    # useful parameters to compare floating point variables to 0 and 1
    epsilon = 1e-4
    local_max = 0.9999


    # implementing a loop that sets to 1 the maximum fractional value of z[i]
    # it stops when n_z_equal_to_1 reaches 'p', at this point no fractional values
    # are present thanks to the constr A4
    n_z_equal_to_1 = 0
    while n_z_equal_to_1 < p:
        z_matrix = np.array([z[i].x if (z[i].x < local_max) else 0 for i in N])
        index = np.unravel_index(np.argmax(z_matrix, axis=None), z_matrix.shape)
        constraint = m.add_constr(z[index[0]] == 1)

        # optimize after every constr is added to improve quality of solution
        status = m.optimize(relax=True)

        # check whether the new constraint makes the problem unfeasible, if so remove it
        if status != mip.OptimizationStatus.OPTIMAL:
            m.remove(constraint)
            m.add_constr(z[index[0]] == 0)
            status = m.optimize(relax=True)
            if status != mip.OptimizationStatus.OPTIMAL:
                print("Optimization failed! Error!")
                break

        # add cuts to medium sized models
        if(n<=80):
            cp = m.generate_cuts(max_cuts=max_c)
            m += cp
            m.optimize(relax = True)

        # count how many variables are equal to 1
        n_z_equal_to_1 = sum([1 if z[i].x >= local_max else 0 for i in N])



    # This loop is looking for fractional values in w similarly to z.
    # For every farm, when fractional values are present, if the max of the row is
    # below epsilon, we set the row to zero, meaning the farm doesn't sell, otherwise
    # we set it to 1 and thanks to A1 the rest of the row goes to 0
    # it may add cuts before re-optimizing again (medium sized models). 
    fractional = True
    while fractional:
        w_matrix = np.array([[w[i, j].x for j in N] for i in N])

        # check for fractional values in the solution
        fractional = False
        for i in N:
            for j in N:
                if (w_matrix[i,j] > epsilon and  w_matrix[i,j] < 1-epsilon):
                    fractional = True
        if not fractional:
            break

        for i in N:
            w_row = np.array([w[i,j].x for j in N])
            if w_row.all() < 1-epsilon:
                if w_row.any() > 0:

                    # add constr only if there are fractional values
                    index = np.unravel_index(np.argmax(w_row, axis=None), w_row.shape)
                    if (w_row[index[0]] < epsilon and w_row[index[0]] > 0) :
                        constraint = m.add_constr(mip.xsum(w[i,j] for j in N) == 0)
                    elif (w_row[index[0]] > epsilon) :
                        constraint = m.add_constr(w[i,index[0]] == 1)

                    # optimize after every constr is added to improve quality of solution
                    status = m.optimize(relax=True)

                    # check whether the new constraint makes the problem unfeasible, if so remove it
                    if status != mip.OptimizationStatus.OPTIMAL:
                        print("Removing constraint")
                        m.remove(constraint)
                        constraint = m.add_constr(mip.xsum(w[i,j] for j in N) == 0)
                        status = m.optimize(relax=True)
                        if status != mip.OptimizationStatus.OPTIMAL:
                            print("Optimization failed! Error!")
                            break

                    if(n<=80):
                        cp = m.generate_cuts(max_cuts=max_c)
                        m += cp
                        m.optimize(relax = True)


    
    # print w, q and z
    tons = [[q[i, j].x for j in N] for i in N]
    sold = [[w[i, j].x for j in N] for i in N]
    farms = [[z[i].x for i in N]]
    """
    print('\ntons sold')
    print('\n'.join(['\t'.join([str(round(cell, 3)) for cell in row]) for row in tons]))
    """
    print('\nif sold')
    print('\n'.join(['\t'.join([str(round(cell, 3)) for cell in row]) for row in sold]))
    
    print('\nfarm with plant')
    print('\n'.join(['\t'.join([str(round(cell, 3)) for cell in row]) for row in farms]))
    

    return m.objective_value

# change only "num_inst" to select number of instance 
num_inst = 1
inst = load_instance("instance_" + str(num_inst) + ".json")
obj = solve(inst)

print("result: {}".format(obj))
if num_inst <= 4:
    res = load_result("instance_" + str(num_inst) + ".txt")
    gap = 100 * (obj - res) / res
    print("expected: {}".format(res))
    print("gap: {}".format(gap))

result: 1324477.6736137536
expected: 1324477.6736137536
gap: 0.0
